### Ali Darabi - 810100264
### Hesam Ramezanian - 810100248
### MohammadReza Alavi - 810100253

# Data Science - Project Phase 1

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import warnings

In [ ]:
df = pd.read_csv('Beijing.csv', encoding='gb18030')
df.info()

In [ ]:
df.head()

# <span style="color:cyan">Preprocessing:</span>

In [ ]:
df = df.drop('url', axis=1)
df = df.drop('id', axis=1)

<span style="color:pink">We drop these columns because they have no use for us.</span>

In [ ]:
df.isnull().sum()

In [ ]:
warnings.filterwarnings("ignore")
missingDf = df.isnull().sum().reset_index()
missingDf.columns = ['Variable', 'value']
missing = missingDf[missingDf['value'] > 0]
missing.loc[missing.index, 'value'] = missing['value'] / len(df)
missing = missing.sort_values(by='value', ascending=False)

fig = px.bar(missing, x='value', y='Variable', orientation='h',
             color='value', color_continuous_scale='Blackbody',
             text='value', text_auto='.2%',
             hover_data={'value': ':.2%'})

fig.update_layout(
    title={
        'text': 'Missing Value Proportion',
        'font': {'family': 'Lato', 'size': 20, 'weight': 'bold'}
    },
    xaxis_title={
        'text': 'Proportion of Missing Values',
        'font': {'family': 'Lato', 'size': 16, 'weight': 'bold'}
    },
    yaxis_title={
        'text': 'Variable',
        'font': {'family': 'Lato', 'size': 16, 'weight': 'bold'}
    },
    font=dict(family='Lato', size=16),
    template='seaborn',
    height=600,
    bargap=0.1
)

fig.update_traces(
    opacity=0.9
)

fig.show()

In [ ]:
df = df.drop('DOM', axis=1)
df = df.dropna(how='any')

<span style="color:pink">As you can see, about 50 percent of the 'DOM' column is missing values, so we drop this column.</span>

<span style="color:pink">Because the number of missing values in other columns is small (under 1 percent), we drop these rows.</span>

In [ ]:
print("Dataset size after dropping missing values: ", len(df))

In [ ]:
df['livingRoom'] = pd.to_numeric(df['livingRoom'], errors='coerce')
df['livingRoom'].unique()

In [ ]:
df['drawingRoom'] = pd.to_numeric(df['drawingRoom'], errors='coerce')
df['drawingRoom'].unique()   

In [ ]:
df['bathRoom'] = pd.to_numeric(df['bathRoom'], errors='coerce')
df['bathRoom'].unique()       

In [ ]:
df['constructionTime'] = pd.to_numeric(df['constructionTime'], errors='coerce')     
df['constructionTime'].unique()

<span style="color:pink">Because in some columns we have string values like '1' mixed with int values, we have to convert these values to numeric values.</span>

In [ ]:
df['constructionTime'] = df['constructionTime'].interpolate(method='linear')
df['constructionTime'] = df['constructionTime'].round()
df['constructionTime'] = df['constructionTime'].astype(int)

<span style="color:pink">In the `constructionTime` column, we have NaN values, so we have to fill them. We use the interpolation method.</span>

In [ ]:
df['tradeTime'] = pd.to_datetime(df['tradeTime'])
df['tradeTimeYear'] = df['tradeTime'].dt.year
df['tradeTimeMonth'] = df['tradeTime'].dt.month
df = df.drop('tradeTime', axis=1)
df['tradeTimeYear'].value_counts()

<span style="color:pink">We convert `tradeTime` to Date and then add two columns `tradeTimeYear` and `tradeTimeMonth`, and then drop the original `tradeTime` column.</span>

In [ ]:
df = df[df['tradeTimeYear'] > 2009]
df['tradeTimeYear'].value_counts()

<span style="color:pink">In this part, we remove outliers from the `tradeTimeYear` column.</span>

In [ ]:
unique = df["floor"].unique()
arr = [re.sub(r'\d+', '', s) for s in unique]
newArr = list(set(arr))
print("Unique chinese words: ", newArr)

In [ ]:
translationDict = {
    '底': 'Bottom',
    '未知': 'Unknown',
    '中': 'Middle',
    '钢混结构': 'Steel-concrete composite structure',
    '高': 'High',
    '顶': 'Top',
    '混合结构': 'Mixed structure',
    '低': 'Low'
}

def Translate(df, col):
    df[col] = df[col].replace(translationDict, regex=True)
    print(col," : ",df[col].unique())

In [ ]:
translateColumns = ['drawingRoom', 'bathRoom', 'floor', 'constructionTime']
for col in translateColumns:
    Translate(df, col)

<span style="color:pink">We have to translate these columns because they are made up of Chinese words.</span>

In [ ]:
df['buildingStructure'].unique()

In [ ]:
df = df[df['buildingStructure'] != 1]
df['buildingStructure'].unique()

<span style="color:pink">We drop rows with unknown values for the `buildingStructure` column because there are only 45 of them.</span>

In [ ]:
print("The coordinates of the lower left corner:", min(df['Lat']), min(df['Lng']))
print("The coordinates of the upper right corner:", max(df['Lat']), max(df['Lng']))

In [ ]:
def extractFloorNumber(floor):
    match = re.search(r'\d+', floor)
    return int(match.group()) if match else None

df['floorNumber'] = df['floor'].apply(extractFloorNumber)
df = df.drop('floor', axis=1)
df['floorNumber'].unique()

<span style="color:pink">We create a new column called `floorNumber` from the `floor` column and then drop the original column.</span>

# <span style="color:cyan">Plots:</span>

In [ ]:
def drawBeijingHousingDistribution(column, color, df):
    img = plt.imread("map1.jpg")
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(img, alpha=0.5, extent=[116.07 ,116.74, 39.62, 40.26], aspect='auto')
    sns.scatterplot(data=df, x="Lng", y="Lat", alpha=0.8, hue=column, palette=color,size='price',sizes=(1,250))
    plt.legend(loc='best')
    plt.xlabel('longtitude', fontsize=18)
    plt.ylabel('latitude', fontsize=18)
    plt.title(f'Beijing housing distribution by {column}', fontsize=24)
    plt.show()

In [ ]:
valueMap = {
    2: "mixed",
    3: "brick and wood",
    4: "brick and concrete",
    5: "steel",
    6: "steel-concrete composite"
}
newDf = df.copy()
newDf["buildingStructure"] = df["buildingStructure"].map(valueMap)
drawBeijingHousingDistribution("buildingStructure", "plasma", newDf)

<span style="color:pink">As you can see, most of the houses are made of steel-concrete composite and mixed, and their distribution is spread on the map.</span>

In [ ]:
data = df['price']
numBins = int(np.sqrt(len(data)))
fig = px.histogram(df, x='price', nbins=numBins, title='Distribution of Housing Prices')
fig.update_layout(xaxis_title='Price', yaxis_title='Count', bargap=0.1)
fig.show()

<span style="color:pink">The histogram highlights the prevalence of lower-priced housing units in Beijing, with a concentration around the 20k-40k range and a steep drop-off in frequency as prices increase, indicating a skewed market favoring affordability over luxury housing.</span>

In [ ]:
df['subway'] = df['subway'].astype(int)
df['subway'].unique()

In [ ]:
valueMap = {
    0: "no Subway",
    1: "has Subway",
}
newDf["subway"] = df["subway"].map(valueMap)
drawBeijingHousingDistribution("subway", "plasma", newDf)

<span style="color:pink">As you can see, most houses near the center of the city have access to the subway.</span>


In [ ]:
drawBeijingHousingDistribution("district", "muted", df)

In [ ]:
sns.displot(df, x='price', hue='district', palette='muted', alpha=1, multiple="stack", height=8, aspect=1.5)
plt.title("Distribution of Prices by District")
plt.show()

<span style="color:pink">As the number of the district increases, the price range decreases, and the number of houses also decreases. The reason for this could be that districts with higher numbers are usually around the city.</span>

In [ ]:
dfNoElevator = df[df['elevator'] == 0]
dfWithElevator = df[df['elevator'] == 1]

In [ ]:
colors = ['#1f77b4', '#ff7f0e']
plt.pie(df['elevator'].value_counts().values, colors=colors,
           wedgeprops=dict(width=.8),autopct='%1.2f%%')
plt.title('percentage of having elevator') 
plt.legend(loc="upper right",
           labels=['with elevator', 'without elevator'])
plt.show()

fig = make_subplots(rows=1, cols=2, shared_xaxes=True, shared_yaxes=True, subplot_titles=('Distribution of Floor Numbers (Elevator = 0)', 'Distribution of Floor Numbers (Elevator = 1)'))
trace1 = go.Histogram(x=dfNoElevator['floorNumber'], nbinsx=20, histnorm='probability density', name='Elevator = 0')
trace2 = go.Histogram(x=dfWithElevator['floorNumber'], nbinsx=20, histnorm='probability density', name='Elevator = 1')
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
fig.update_layout(
    title='Floor Number Distribution',
    xaxis1_title='Floor Number',
    xaxis2_title='Floor Number',
    yaxis1_title='Frequency',
    yaxis2_title='Frequency',
    bargap=0.1,
    height=500,
    width=1000,
    xaxis1=dict(tickmode='linear', tick0=0, dtick=5),
    xaxis2=dict(tickmode='linear', tick0=0, dtick=5)
)
fig.show()

<span style="color:pink">As you can see in the plots, houses with higher `floorNumber` have elevators, and most houses with no elevator are under 10 floors.</span>

In [ ]:
trace1 = go.Bar(
    x=df['tradeTimeYear'].value_counts().index,
    y=df['tradeTimeYear'].value_counts().values,
    marker=dict(color='#4CAF50'),  
    text=df['tradeTimeYear'].value_counts().values,
    textposition='auto'
)
trace2 = go.Bar(
    x=df['tradeTimeMonth'].value_counts().index,
    y=df['tradeTimeMonth'].value_counts().values,
    marker=dict(color='#2196F3'), 
    text=df['tradeTimeMonth'].value_counts().values,
    textposition='auto'
)
fig = make_subplots(rows=1, cols=2, subplot_titles=['Number of Transactions by Year', 'Number of Transactions by Month'])
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
fig.update_layout(
    title='Transactions over Time',
    height=600,
    width=1200,
    bargap=0.1,
    xaxis1_tickmode='linear', 
    xaxis1_tickvals=sorted(df['tradeTimeYear'].unique()),
    xaxis1_title='Year',
    xaxis2_title='Month',
    yaxis1_title='Number of Transactions',
    yaxis2_title='Number of Transactions'
)
fig.show()

<span style="color:pink">As you can see, in the third month, we have a higher number of transactions, and after the year 2016, the number of transactions greatly decreased.</span>

In [ ]:
def plotOverAvgTotalPrice(columnName):
    AvgPrice = df.groupby(columnName)['totalPrice'].mean().reset_index()
    AvgPrice = AvgPrice.rename(columns={'totalPrice': 'AvgTotalPrice'})
    fig = px.line(AvgPrice, x=columnName, y='AvgTotalPrice', title=f'Average Price Over {columnName}')
    fig.show()

In [ ]:
plotOverAvgTotalPrice('tradeTimeYear')

<span style="color:pink">As you can see on the plot, as time increases, the average price also increases.</span>

In [ ]:
corrMatrix = df.corr()
plt.figure(figsize=(30, 15))
mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
sns.heatmap(corrMatrix, annot=True, cmap='coolwarm', mask=mask)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
plotOverAvgTotalPrice('constructionTime')

In [ ]:
df['constructionTime'].value_counts()

<span style="color:pink">In the Beijing housing market, properties built in a shorter timeframe tend to have higher average prices, while those with longer construction periods generally have lower average prices, with some exceptions. However, the scarcity of data points for older houses with longer construction times may affect the reliability of the average price calculations for those construction time durations.</span>

In [ ]:
df['fiveYearsProperty'] = df['fiveYearsProperty'].astype(int)
valueMap = {
    0: "less than 5 years",
    1: "more than 5 years",
}
newDf["fiveYearsProperty"] = df["fiveYearsProperty"].map(valueMap)
drawBeijingHousingDistribution("fiveYearsProperty", "muted", newDf)

<span style="color:pink">As you can see, houses whose owners have had the property for less than 5 years or more than 5 years are spread all over the map, and we cannot distinguish them.</span>

In [ ]:
def plotRoomsOverAvgSquare(columnName):
    avgSquare = df.groupby(columnName)['square'].mean().reset_index()
    avgSquare = avgSquare.rename(columns={'square': 'Avgsquare'})
    fig = px.line(avgSquare, x=columnName, y='Avgsquare', title=f'Average square Over {columnName}')
    fig.show()

In [ ]:
plotRoomsOverAvgSquare('livingRoom')

In [ ]:
plotRoomsOverAvgSquare('drawingRoom')          

In [ ]:
plotRoomsOverAvgSquare('bathRoom')  

In [ ]:
plotRoomsOverAvgSquare('kitchen') 

<span style="color:pink">As you can see in the plots, as the number of rooms or kitchens increases, the average square footage of houses also increases.</span>